LIBRERIE
- https://pyomo.readthedocs.io/en/stable/index.html
- https://simpy.readthedocs.io/en/latest/

In [2]:
import simpy

In [42]:
# https://simpy.readthedocs.io/en/latest/simpy_intro/process_interaction.html

import simpy
SIM_TIME = 40
parks = 0
drives =0
charges=0

def car(env):
    global parks, drives ,charges  #per aggiornare variabili globali da dentro uan funzione 
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        parks = parks +1 
        yield env.timeout(parking_duration) #function to create a Timeout event (wait) 

        print('Start driving at %d' % env.now)
        trip_duration = 2
        drives =drives+1
        yield env.timeout(trip_duration) 
        
        print('Start recharge at %d' % env.now)
        charges_duration = 7
        charges =charges+1
        yield env.timeout(charges_duration) #function to create a Timeout event (wait) 
        
env = simpy.Environment()
env.process(car(env))
env.run(until=SIM_TIME)
print(parks,drives,charges)


Start parking at 0
Start driving at 5
Start recharge at 7
Start parking at 14
Start driving at 19
Start recharge at 21
Start parking at 28
Start driving at 33
Start recharge at 35
3 3 3


In [66]:
# https://simpy.readthedocs.io/en/latest/simpy_intro/process_interaction.html

import simpy

class Car(object):
    def __init__(self, env):
        self.env = env         # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
       while True:
           print('Start parking and charging at %d' % self.env.now)
           charge_duration = 5 # We may get interrupted while charging the battery
           try:
               yield self.env.process(self.charge(charge_duration))
           except simpy.Interrupt:                # When we received an interrupt, we stop charging and switch to the "driving" state
               print('Was interrupted. Hope, the battery is full enough ...')

           print('Start driving at %d' % self.env.now)
           trip_duration = 2
           yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

    def driver(env, car):
        yield env.timeout(3)
        car.action.interrupt()
    
env = simpy.Environment()
car1 = Car(env)
# car2 = Car(env)
env.run(until=15)


Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


In [4]:
# Carwash example.
# Covers:
# - Waiting for other processes
# - Resources: Resource
# Scenario:
#   A carwash has a limited number of washing machines and defines a washing processes that takes some (random) time.
#   Car processes arrive at the carwash at a random time. If one washing machine is available, they start the washing process and wait for it
#   to finish. If not, they wait until they an use one.

import random
import simpy

RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME = 5      # Minutes it takes to clean a car
T_INTER = 3       # Create a car every ~7 minutes
SIM_TIME = 20     # Simulation time in minutes

class Carwash(object):
#     carwash has a limited number of machines (NUM_MACHINES) to clean cars in parallel.  Cars have to request one of the machines. When they got one, they can start the washing processes and wait for it to finish (which     takes `washtime` minutes).
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
#     The washing processes. It takes a ``car`` processes and tries to clean it
        yield self.env.timeout(WASHTIME)
        print("Carwash removed %d%% of %s's dirt." %
              (random.randint(50, 99), car))

def car(env, name, cw):
#     """The car process (each car has a ``name``) arrives at the carwash (``cw``) and requests a cleaning machine.
#     It then starts the washing process, waits for it to finish and leaves to never come back ...
    print('%s arrives at the carwash at %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s enters the carwash at %.2f.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s leaves the carwash at %.2f.' % (name, env.now))

def setup(env, num_machines, washtime, t_inter):
#     Create a carwash, a number of initial cars and keep creating cars approx. every ``t_inter`` minutes."""
    carwash = Carwash(env, num_machines, washtime)   # Create carwash
    for i in range(4):    
        env.process(car(env, 'Car %d' % i, carwash))   # Create 4 initial cars
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Car %d' % i, carwash))      # Create more cars while the simulation is running

# Setup and start the simulation
print('Carwash')
random.seed(RANDOM_SEED)  # This helps reproducing the results

env = simpy.Environment()   # Create an environment 
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))  #start the setup process
env.run(until=SIM_TIME) # Execute!

Carwash
Car 0 arrives at the carwash at 0.00.
Car 1 arrives at the carwash at 0.00.
Car 2 arrives at the carwash at 0.00.
Car 3 arrives at the carwash at 0.00.
Car 0 enters the carwash at 0.00.
Car 1 enters the carwash at 0.00.
Car 4 arrives at the carwash at 1.00.
Car 5 arrives at the carwash at 2.00.
Carwash removed 65% of Car 0's dirt.
Carwash removed 64% of Car 1's dirt.
Car 6 arrives at the carwash at 5.00.
Car 0 leaves the carwash at 5.00.
Car 1 leaves the carwash at 5.00.
Car 2 enters the carwash at 5.00.
Car 3 enters the carwash at 5.00.
Car 7 arrives at the carwash at 7.00.
Car 8 arrives at the carwash at 8.00.
Carwash removed 55% of Car 2's dirt.
Carwash removed 87% of Car 3's dirt.
Car 2 leaves the carwash at 10.00.
Car 3 leaves the carwash at 10.00.
Car 4 enters the carwash at 10.00.
Car 5 enters the carwash at 10.00.
Car 9 arrives at the carwash at 13.00.
Carwash removed 52% of Car 4's dirt.
Carwash removed 51% of Car 5's dirt.
Car 4 leaves the carwash at 15.00.
Car 5 leav